In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, SimpleRNN

# import nltk
# nltk.download('stopwords')

# Data Loading And Dropping irrelevant Values

In [2]:
df = pd.read_csv('amazon_reviews.csv')
df = df.drop('review_score', axis=1)

df = df.dropna();

print(df)

      sentiments                                     cleaned_review  \
0       positive  i wish would have gotten one earlier love it a...   
1        neutral  i ve learned this lesson again open the packag...   
2        neutral          it is so slow and lags find better option   
3        neutral  roller ball stopped working within months of m...   
4        neutral  i like the color and size but it few days out ...   
...          ...                                                ...   
17335   positive  i love this speaker and love can take it anywh...   
17336   positive  i use it in my house easy to connect and loud ...   
17337   positive  the bass is good and the battery is amazing mu...   
17338   positive                                            love it   
17339    neutral                                       mono speaker   

       cleaned_review_length  
0                         19  
1                         88  
2                          9  
3                      

# Data Preprocessing

In [3]:
stopwords = stopwords.words('english')

remove_stopwords_regex = r'\b(' + r'|'.join(stopwords) + r')\b\s*'

df['cleaned_review'] = df.apply(lambda x : re.sub(remove_stopwords_regex, '', x['cleaned_review']), axis=1)


# Word Embedding

## Generating Unique Tokens

In [4]:
tokens = []
cleaned_reviews = df['cleaned_review']

tokens = [word for review in cleaned_reviews for word in review.split(' ')]
tokens = list(set(tokens))
tokens.remove('')

print(tokens)

['cleaned', 'trucks', 'academic', 'vc', 'percussion', 'diagonal', 'relief', 'best', 'generations', 'bleeding', 'gifted', 'thankful', 'conspros', 'threaded', 'remains', 'mac', 'swiftly', 'cloud', 'tote', 'blows', 'amazon', 'scammed', 'donate', 'source', 'sis', 'boyfriend', 'baught', 'mood', 'kindle', 'thinking', 'amazondo', 'receptacle', 'combination', 'disconnecting', 'wording', 'superrrrrrrrrr', 'bout', 'processing', 'services', 'burned', 'grad', 'homework', 'tie', 'lovin', 'notifying', 'liars', 'nigeria', 'bean', 'agree', 'backed', 'servicekeyboard', 'spazzes', 'mst', 'facing', 'sleekness', 'cancelled', 'forgiven', 'shoddy', 'influence', 'brand', 'slack', 'heat', 'benefit', 'dear', 'bueno', 'bur', 'clarify', 'wantedit', 'minimum', 'flexes', 'contour', 'seatbelt', 'corrected', 'butfulake', 'indication', 'reaching', 'padded', 'pure', 'qualitylove', 'gouging', 'following', 'troubles', 'kidding', 'perfecf', 'separations', 'elitebook', 'nba', 'interface', 'afford', 'unhooked', 'showers', 

## Converting reviews to sequences

In [5]:
word_to_index = {word: index for index, word in enumerate(tokens,1)}

def review_to_indices(review):
    return [word_to_index.get(word, 0) for word in review.split(' ')]

df['indexed_review'] = df.apply(lambda x : review_to_indices(x['cleaned_review']), axis=1)
max_length = max([len(review) for review in df['indexed_review']])

## Padding Sequences

In [6]:
max_length = 50
padded_sequences = pad_sequences(df['indexed_review'], maxlen=max_length, padding='post')

# Splitting Data and encoding labels

In [7]:
X = padded_sequences
y = df['sentiments']

In [8]:
# encoding the target variable
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [9]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Building LSTM Model

In [10]:
# Building the model
model = Sequential()
model.add(Embedding(len(tokens)+1, 128, input_length=max_length))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [11]:
# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10


380/380 [==============================] - 14s 25ms/step - loss: 0.5934 - accuracy: 0.7504
Epoch 2/10
380/380 [==============================] - 9s 24ms/step - loss: 0.3519 - accuracy: 0.8715
Epoch 3/10
380/380 [==============================] - 9s 25ms/step - loss: 0.2517 - accuracy: 0.9139
Epoch 4/10
380/380 [==============================] - 10s 26ms/step - loss: 0.1869 - accuracy: 0.9402
Epoch 5/10
380/380 [==============================] - 10s 26ms/step - loss: 0.1507 - accuracy: 0.9534
Epoch 6/10
380/380 [==============================] - 10s 26ms/step - loss: 0.1194 - accuracy: 0.9637
Epoch 7/10
380/380 [==============================] - 10s 26ms/step - loss: 0.0991 - accuracy: 0.9715
Epoch 8/10
380/380 [==============================] - 9s 25ms/step - loss: 0.0868 - accuracy: 0.9758
Epoch 9/10
380/380 [==============================] - 10s 25ms/step - loss: 0.0714 - accuracy: 0.9791
Epoch 10/10
380/380 [==============================] - 9s 25ms/step - loss: 0.0698 

# LSTM Model Accuracy

In [12]:
# Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

163/163 [==============================] - 2s 7ms/step - loss: 0.5992 - accuracy: 0.8585
Loss: 0.5991509556770325, Accuracy: 0.8585159778594971


# LSTM Model Summary

In [13]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 128)           1209344   
                                                                 
 bidirectional (Bidirection  (None, 50, 128)           98816     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 50, 128)           0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 3)                 1

In [14]:
# Save the model
# model.save('lstm.h5')

# Building RNN Model

In [15]:
# Building the model
model = Sequential()
model.add(Embedding(len(tokens)+1, 128, input_length=max_length))
model.add(Bidirectional(SimpleRNN(64, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(SimpleRNN(32)))
model.add(Dropout(0.3))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Training the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
380/380 [==============================] - 10s 19ms/step - loss: 0.6916 - accuracy: 0.7171
Epoch 2/10
380/380 [==============================] - 7s 19ms/step - loss: 0.3972 - accuracy: 0.8565
Epoch 3/10
380/380 [==============================] - 7s 18ms/step - loss: 0.2110 - accuracy: 0.9307
Epoch 4/10
380/380 [==============================] - 7s 18ms/step - loss: 0.1337 - accuracy: 0.9582
Epoch 5/10
380/380 [==============================] - 7s 18ms/step - loss: 0.0775 - accuracy: 0.9767
Epoch 6/10
380/380 [==============================] - 7s 19ms/step - loss: 0.0739 - accuracy: 0.9776
Epoch 7/10
380/380 [==============================] - 7s 19ms/step - loss: 0.0574 - accuracy: 0.9837
Epoch 8/10
380/380 [==============================] - 7s 18ms/step - loss: 0.0564 - accuracy: 0.9829
Epoch 9/10
380/380 [==============================] - 7s 18ms/step - loss: 0.1021 - accuracy: 0.9660
Epoch 10/10
380/380 [==============================] - 7s 19ms/step - loss: 0.0559 - accur

# RNN Model Accuracy

In [17]:
# Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

163/163 [==============================] - 1s 4ms/step - loss: 0.7725 - accuracy: 0.8306
Loss: 0.772497832775116, Accuracy: 0.83064204454422


# RNN Model Summary

In [18]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 128)           1209344   
                                                                 
 bidirectional_2 (Bidirecti  (None, 50, 128)           24704     
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 50, 128)           0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                10304     
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                

In [19]:
# model.save('rnn.h5')